# EnergyPlus Example

In [1]:
import os
from zerobnl import CoSim

You can safely ignore the following error (it will also be in the nodes logs):

RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88

-> [Numpy documentation](https://github.com/numpy/numpy/pull/432)

In [2]:
sim = CoSim()

# There are 2 environments in this example: a python-based one and an EnergyPlus-based one.
sim.create_meta_model("MetaBase", [("a", "Cdeg")], [("b", "Cdeg")])
sim.create_environment("EnvBase", "wrapper_base.py", "Dockerfile_base")

sim.create_meta_model("MetaEplus", [("Ts_second", "Cdeg")], [("Tr_second", "Cdeg")])
sim.create_environment("EnvEplus", "wrapper_eplus.py", "Dockerfile_na")

# There is one model per each environment in this example. 
# OBS!! The name Base# indicates a model. Each model is contained in a node. There can be more nodes in each environment.
sim.add_node("Base0", "MetaBase", "EnvBase", init_values={})
sim.add_node("Base1", "MetaEplus", "EnvEplus", init_values={"Tr_second": 45})

# Links among nodes
sim.add_link("Base0", "b", "Base1", "Ts_second") # get from Base0 and set to Base1
sim.add_link("Base1", "Tr_second", "Base0", "a") # get from Base1 and set to Base0

# Create groups from the simulation sequence. Nodes in the same group run in parallel. 
# A group is defined within the first level of square brackets.
sim.create_sequence([["Base0"], ["Base1"]])
sim.set_time_unit("seconds")
sim.create_steps([3600] * 24)

INFO :: Meta-model BaseMeta created.
INFO :: Meta-model EplusMeta created.
INFO :: Model BaseModel created.
INFO :: Model EplusModel created.
INFO :: Node Base0 created.
INFO :: Node Base1 created.
INFO :: Link created Base0 -> Base1.
INFO :: Link created Base1 -> Base0.
INFO :: The group ('Base0',) have been created.
INFO :: The group ('Base1',) have been created.
INFO :: The sequence [['GRP0', ('Base0',)], ['GRP1', ('Base1',)]] have been created.
INFO :: 24 steps have been created.


Once the next step has been launched, logging `INFO :: Waiting for local nodes to run..`, you need to run tho following command `wrapper_eplus.py Base1 GRP1` in the indicated folder (in a dedicated environment) in order to run the local node.

In [3]:
sim.run()

INFO :: Starting the simulation's deployment...
INFO :: Starting simulation...
INFO :: Waiting for local nodes to run...
INFO :: Simulation finished in 0 min and 18 sec


If you see `INFO :: Simulation finished in X min and Y sec` it means everything went well.
You can find logs of the nodes in the file `nodes.log`, it's a text file you can open it directly in Jupyter or in your favorite text editor.

At the begining of the file you will find a serie of:

`Step X/10 : DO SOMETHING
 ---> 29d2f3226daf`
 
It's the logs of the creation of the Docker image, based on the provided Dockerfile (here `Dockerfile_base`).

Then all the logs are structures in the same way:

`<node>    | <level> :: <message>`

* `node` refers to the concerned simulation node or orchestrator
* `level` can be `DEBUG`: used for development purpose, `INFO`: giving you info on the running process, `WARNING`: warning you on action to make or some weird behaviour, `ERROR`: something went wrong and `CRITICAL`: something went really wrong.
* `message` is the body of the log, it describes what's happening.

You can also find information on the ongoing simulation in the file `activity.log` (in the root folder for the main processus and on the temporary folder for each node)

In [4]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

INFO :: Connected to Redis DB


,IN/OUT,Node,Attribute
0,OUT,Base0,b
1,IN,Base1,Ts_second
2,OUT,Base1,Tr_second
3,IN,Base0,a


The name to the stored results are build as `<type>||<node>||<attribute>`.

`type` can be:
* `IN` if it's an input attribute (to set - stored automatically)
* `OUT` if it's an output attribute (to get - stored automatically)
* `X` if it's an internal value (stored by the user, using the `save_attribute()` method in the wrapper)

Knowing this, you can create matching pattern using `*` in order to properly select results.

In [5]:
for key, value in sim.get_results_by_pattern("OUT*Base0*").items():
    print(key)
    print(value)   

INFO :: Matching results: Base0 - a


2000-01-01 01:00:00    0.0
2000-01-01 02:00:00    0.0
2000-01-01 03:00:00    0.0
2000-01-01 04:00:00    0.0
2000-01-01 05:00:00    0.0
2000-01-01 06:00:00    0.0
2000-01-01 07:00:00    0.0
2000-01-01 08:00:00    0.0
2000-01-01 09:00:00    0.0
2000-01-01 10:00:00    0.0
2000-01-01 11:00:00    0.0
2000-01-01 12:00:00    0.0
2000-01-01 13:00:00    0.0
2000-01-01 14:00:00    0.0
2000-01-01 15:00:00    0.0
2000-01-01 16:00:00    0.0
2000-01-01 17:00:00    0.0
2000-01-01 18:00:00    0.0
2000-01-01 19:00:00    0.0
2000-01-01 20:00:00    0.0
2000-01-01 21:00:00    0.0
2000-01-01 22:00:00    0.0
2000-01-01 23:00:00    0.0
dtype: float64


In [6]:
for key, value in sim.get_results_by_pattern("IN*Base0*").items():
    print(key)
    print(value)

INFO :: Matching results: Base1 - Ts_second


2000-01-01 01:00:00    0.0
2000-01-01 02:00:00    0.0
2000-01-01 03:00:00    0.0
2000-01-01 04:00:00    0.0
2000-01-01 05:00:00    0.0
2000-01-01 06:00:00    0.0
2000-01-01 07:00:00    0.0
2000-01-01 08:00:00    0.0
2000-01-01 09:00:00    0.0
2000-01-01 10:00:00    0.0
2000-01-01 11:00:00    0.0
2000-01-01 12:00:00    0.0
2000-01-01 13:00:00    0.0
2000-01-01 14:00:00    0.0
2000-01-01 15:00:00    0.0
2000-01-01 16:00:00    0.0
2000-01-01 17:00:00    0.0
2000-01-01 18:00:00    0.0
2000-01-01 19:00:00    0.0
2000-01-01 20:00:00    0.0
2000-01-01 21:00:00    0.0
2000-01-01 22:00:00    0.0
2000-01-01 23:00:00    0.0
dtype: float64
